In [1]:
from data_utils import get_CIFAR10_data
import numpy as np

import keras
import tensorflow as tf
from keras.layers import Input, Conv2D, Dense, Flatten, Add, ZeroPadding2D, Merge, add, MaxPool2D
from keras.models import Model
from keras.utils.np_utils import to_categorical 
import tensorflow_weave
reload(tensorflow_weave)
from tensorflow_weave import *

/Users/Tim/anaconda2/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
#Load the (preprocessed) CIFAR10 data.

data = get_CIFAR10_data()
for k, v in data.iteritems():
  print '%s: ' % k, v.shape

X_val:  (1000, 3, 32, 32)
X_train:  (49000, 3, 32, 32)
X_test:  (1000, 3, 32, 32)
y_val:  (1000,)
y_train:  (49000,)
y_test:  (1000,)


In [3]:
filter_size = 3
num_filters = 2
pad_size = filter_size//2


inputs = Input(shape=(3,32,32))
num_images = tf.shape(inputs)[0]

x_pad = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)

x_per = Conv2D(num_filters,
               kernel_size = (3,3),
               strides=(1,1),
               padding='valid',
               activation='relu')(x_pad)
x_loc = Conv2D(num_filters,
               kernel_size= (3,3),
               strides=(1,1),
               padding='valid',
               activation = 'relu')(x_pad)

x_zero = ZeroWeave()(x_loc)
x_weave = ArrayWeave()(x_per)
x = Add()([x_weave, x_zero])



x = Conv2D(num_filters,
           kernel_size= (3,3),
           strides=(3,3),
           padding='valid',
           activation = 'relu')(x)

x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(100, activation = 'relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model = Model(inputs=[inputs], outputs=predictions)
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 32, 32)    0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 3, 34, 34)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 2, 32, 32)    56          zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 2, 32, 32)    56          zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
array_weav

In [7]:
num_train = 40000
X_train = data['X_train'][:num_train]
Y_train = data['y_train'][:num_train]  

Y_train = to_categorical(Y_train, num_classes=10)

In [8]:
model.fit(X_train,
          Y_train,
          epochs = 10,
          verbose=1)

Epoch 1/10
40000/40000 [==============================] - 191s 5ms/step - loss: 2.0583 - acc: 0.2801
Epoch 2/10
40000/40000 [==============================] - 161s 4ms/step - loss: 1.7717 - acc: 0.3630
Epoch 3/10
40000/40000 [==============================] - 150s 4ms/step - loss: 1.6545 - acc: 0.4085
Epoch 4/10
40000/40000 [==============================] - 150s 4ms/step - loss: 1.5624 - acc: 0.4388
Epoch 5/10
40000/40000 [==============================] - 146s 4ms/step - loss: 1.4901 - acc: 0.4686
Epoch 6/10
 3328/40000 [=>............................] - ETA: 2:22 - loss: 1.3796 - acc: 0.5027

KeyboardInterrupt: 

In [9]:
filter_size = 3
num_filters = 2
pad_size = filter_size//2


inputs = Input(shape=(3,32,32))
num_images = tf.shape(inputs)[0]

x = ZeroPadding2D(padding=(pad_size,pad_size))(inputs)

x = Conv2D(num_filters,
               kernel_size = (3,3),
               strides=(1,1),
               padding='valid',
               activation='relu')(x)

x = ZeroPadding2D(padding=(pad_size,pad_size))(x)

x = Conv2D(num_filters,
               kernel_size= (3,3),
               strides=(1,1),
               padding='valid',
               activation = 'relu')(x)

x = Conv2D(num_filters,
           kernel_size= (3,3),
           strides=(1,1),
           padding='valid',
           activation = 'relu')(x)

x = MaxPool2D()(x)
x = Flatten()(x)
x = Dense(100, activation = 'relu')(x)
predictions = Dense(10, activation='softmax')(x)

# This creates a model that includes
# the Input layer and three Dense layers
model2 = Model(inputs=[inputs], outputs=predictions)
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 3, 32, 32)         0         
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 3, 34, 34)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 2, 32, 32)         56        
_________________________________________________________________
zero_padding2d_3 (ZeroPaddin (None, 2, 34, 34)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 2, 32, 32)         38        
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 2, 30, 30)         38        
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 2, 15, 15)         0         
__________

In [10]:
model2.fit(X_train,
          Y_train,
          epochs = 10,
          verbose=1)

Epoch 1/10
40000/40000 [==============================] - 85s 2ms/step - loss: 2.2140 - acc: 0.2563
Epoch 2/10
40000/40000 [==============================] - 82s 2ms/step - loss: 1.6871 - acc: 0.4047
Epoch 3/10
40000/40000 [==============================] - 86s 2ms/step - loss: 1.5409 - acc: 0.4551
Epoch 4/10
40000/40000 [==============================] - 89s 2ms/step - loss: 1.4563 - acc: 0.4841
Epoch 5/10
23168/40000 [================>.............] - ETA: 39s - loss: 1.3851 - acc: 0.5112

KeyboardInterrupt: 